In [1]:
 import mysql.connector 

In [2]:
#create a list of all airline ids for later on
airlines= [56377143, 106062176,18332190, 22536055,124476322,26223583,2182373406,38676903,1542862735,253340062,218730857,
          45621423,20626359]

In [3]:
db = mysql.connector.connect(host="127.0.0.1", user="root", password="",database="dbl")
cursor=db.cursor()

In [4]:
#create the dayone view
cursor.execute("CREATE VIEW `dayone` AS SELECT * FROM `tweets` t WHERE t.timestamp < 1558606800000")

ProgrammingError: 1050 (42S01): Table 'dayone' already exists

In [4]:
cursor.execute("SELECT * FROM `dayone` ORDER BY timestamp DESC") 
dayone=cursor.fetchall() #store all tweets of one day from new to old in dayone

In [5]:
cursor.execute("SELECT * FROM `tweets` ORDER BY timestamp DESC")
total=cursor.fetchall() #store all tweets from new to old in total

In [6]:
cursor.execute("SELECT COUNT(*) FROM `tweets` t")
tot_numb=cursor.fetchall()[0][0]
print(tot_numb) #return the total number of tweets

6094135


for all tweet_ids <br>
&emsp; if id in dictionary <br>
&emsp; &emsp; link this to those conversations in dictionary<br>
&emsp; &emsp; update the info for those conversations<br>
&emsp; else<br>
&emsp; &emsp; create a new conversation with values from this tweet<br>
&emsp; &emsp; create a tuple linking this tweet to this conversation<br>
&emsp; if in_reply_to_id not zero<br>
&emsp; &emsp; add conversation id from this tweet to that id's dictionary list<br>

CREATE TABLE table_name (<br>
    column1 datatype,<br>
    column2 datatype,<br>
    column3 datatype,<br>
   ....<br>
); 

In [7]:
def create_tables():
    """This function will create a new table in the MySQL environment called conversations with the following attributes:
    conversation ID, conversation start time, conversation end time, has an airline participated, length of conversations,
    and the response time. Furthermore, also create a table that links tweets to conversations they are a part of."""
    
    cursor.execute("""CREATE TABLE `dbl`.`conversations`(id MEDIUMINT NOT NULL, Tstart BIGINT NOT NULL, Tend BIGINT NOT NULL,
    Airline VARCHAR(50) NOT NULL DEFAULT '0',Length SMALLINT NOT NULL, PRIMARY KEY (id))""")
    
    cursor.execute("""CREATE TABLE `dbl`.`part_of`(tID BIGINT NOT NULL, cID MEDIUMINT NOT NULL, PRIMARY KEY(tID, cID), 
    FOREIGN KEY (tID) REFERENCES `tweets`(id), FOREIGN KEY (cID) REFERENCES `conversations`(id))""")
    return 'done'

def drop_tables():
    """Drop the table called conversations and the one called part_of"""
    cursor.execute("DROP TABLE `part_of`")
    cursor.execute("DROP TABLE `conversations`")

    return 'dropped'

def clear_tables():
    """Clears all values from the tables part_off and conversations"""
    cursor.execute("DELETE FROM `part_of`")
    cursor.execute("DELETE FROM`conversations`")
    return db.commit()

In [16]:
drop_tables()

'dropped'

In [18]:
create_tables()

'done'

INSERT INTO table(c1,c2,...) <br>
VALUES <br>
   (v11,v12,...), <br>
   (v21,v22,...), <br>
    ... <br>
   (vnn,vn2,...); <br>

In [64]:
def fill_tables (tweets: list) -> None:
    """This function will insert into the tables 'part_of' and 'conversations' the correct information given an input
        SQL table tweets.
        """
    tracker = 0
    
    #store what conversations a certain tweet_id will have to become a member of, because a reply to it is in it, in this dict.
    member_of = {}

    #increment this whenever you create a new conversation, to ensure there are no duplicate ids, we start with cID=1
    conv_id = 1
    
    for t in tweets:
        tweet = t[0]
        reply_id = t[2]
        tstamp = t[3]
        user_id = t[5]
        if tweet in member_of: #we have found replies to this tweet before
            convs = member_of.pop(tweet)
            for conv in convs:
                cursor.execute(f"INSERT INTO `part_of`(tID,cID) VALUES ({tweet},{conv})")
                
                cursor.execute(f"SELECT * FROM `conversations` WHERE id={conv}") 
                conv_info=cursor.fetchall() #store the information about the current conversation in conv_info
                conv_air = conv_info[0][3] #the airlines that are participating in this conversation
                length = conv_info[0][4] #the current length of this conversations
                conv_air_lst = list(map(int,conv_air.split(","))) #turn the string into a list of integers
                
                if user_id in airlines:    #checks if the user is an airline and, if yes, stores which airlines
                                            #have participated.
                    if conv_air_lst == [0]: #if or no airline has participated in this conversation.
                        cursor.execute(f"""UPDATE `conversations` SET Tstart={tstamp}, Airline = {user_id}, length = {length+1}
                        WHERE id={conv}""")
                    
                    elif user_id not in conv_air_lst:#if this airline has not yet participated, but another one has
                        conv_air_new =  conv_air_lst+ [user_id] #store the airlines already participating
                                                                #and the new one in a list
                        for line in range(len(conv_air_new)): #Update the table with new list.
                            if line == 0: #if its the first airlin, no space needed
                                str_air=str(conv_air_new[line])
                            else:
                                str_air = str_air+ "," +str(conv_air_new[line])
                                
                        cursor.execute(f"""UPDATE `conversations` SET Tstart={tstamp}, Airline = '{str_air}',
                        length = {length+1} WHERE id={conv}""")
                        print("happened")
                else:
                    cursor.execute(f"UPDATE `conversations` SET Tstart={tstamp}, length = {length+1} WHERE id={conv}")
            tracker += 1
            
        else: #this is the last tweet of a conversation
            convs=[conv_id] #used for later if statement.
            
            if user_id in airlines: #is the user_id of this tweet an airline
                cursor.execute(f"""INSERT INTO `conversations`(id,Tstart, Tend, Airline, length)
                VALUES ({conv_id}, {tstamp}, {tstamp}, {user_id}, 1)""") #create conversation in table
            
            else: #this tweet is not from an airline
                cursor.execute(f"""INSERT INTO `conversations`(id,Tstart, Tend, length) 
                VALUES ({conv_id}, {tstamp}, {tstamp}, 1)""") #create conversation in table
            
            cursor.execute(f"INSERT INTO `part_of`(tID,cID) VALUES ({tweet},{conv_id})")
            conv_id += 1 #increment the conversation id to ensure no duplicate conversation ids occur
        
        if reply_id != 0: #if this tweet replies to someone
            
            if reply_id in member_of:   #if the tweet this replies to already has another reply, add the new conversations to dict
                                    #and keep old one.
                member_of[reply_id] = member_of[reply_id] + convs
                
            else:
                member_of[reply_id] = convs        
    return db.commit()#conv_id, tracker#,member_of

In [ ]:
FT_tot = fill_tables(total)


In [ ]:
if sum(FT_tot) == tot_numb:
    print("Everything worked correctly")

In [65]:
clear_tables()

In [66]:
#try to run the function for dayone only
FT_dayone = fill_tables(dayone)
db.commit()

In [ ]:
%%timeit
fill_tables(total)
clear_tables()
print('done')

total: <br>
attempt 1:

dayone: <br>
attempt 1:The slowest run took 13.52 times longer than the fastest. This could mean that an intermediate result is being cached. 6min 45s ± 6min 58s per loop (mean ± std. dev. of 7 runs, 1 loop each) <br>
attempt 2: 2.25 s ± 39.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each <br>
attempt 3: 2.28 s ± 49.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) <br>
attempt 4: 2.37 s ± 70.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) <br>
attempt 5: 2.38 s ± 45.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) <br>
attempt 6: 2.38 s ± 53.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)